In [307]:
import imp
imp.reload(extractor)


<module 'extractor.extractor' from '/Users/oosv/prototype/extractor/extractor.py'>

In [313]:
from extractor import extractor
import numpy as np
import os
path_en = 'data/voxforge/english'
file_list = os.listdir(path_en)

for f in range(len(file_list)):
    file_list[f] = path_en + '/' + file_list[f]
'''
extracts 0.75s frames of features
these are in a 3,75,13 ndarray
3 - mfcc, delta, deltadelta
75 - 75 25ms frames with a step of 10ms
13 - cepstral features
'''
features_en = extractor.make_feature_set(file_list) 


/Users/oosv/miniconda3/lib/python3.7/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


In [318]:
print(np.shape(features_en[2]))
print(f"total samples:\t{len(features_en)}")
print(f"total hours english:\t{len(features_en) * 0.75 / 60}")

(3, 75, 13)
total samples:	65917
total hours:	823.9625


In [319]:
path_es = 'data/voxforge/spanish'
file_list = os.listdir(path_es)

for f in range(len(file_list)):
    file_list[f] = path_es + '/' + file_list[f]

features_es = extractor.make_feature_set(file_list) 


/Users/oosv/miniconda3/lib/python3.7/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


In [322]:
print(np.shape(features_es[2]))
print(f"total samples:\t{len(features_es)}")
print(f"total hours spanish:\t{len(features_es) * 0.75 / 60}")
print(f"total hours combined:\t{(len(features_es) + len(features_en)) * 0.75/60}")

(3, 75, 13)
total samples:	23472
total hours spanish:	293.4
total hours combined:	1117.3625


In [208]:
from sklearn.mixture import GaussianMixture
gmm_en = GaussianMixture(n_components=30, covariance_type='full', max_iter=100, random_state=0)
train_en = features_en[:int(len(features_en) * 3 / 4)]
sample_vectors = []
for vec in train_en:
    vec = np.swapaxes(vec,0,1)
    if not len(vec) < 75:
        vec = vec.reshape(75, 39)
        sample_vectors.append(vec)
shape = np.shape(sample_vectors)
sample_vectors = np.array(sample_vectors).reshape((shape[0] * shape[1]), shape[2])
gmm_en = gmm_en.fit(sample_vectors)
    

In [167]:
print(len(features_en))
print(len(features_es))

65916
23472


In [219]:
gmm_es = GaussianMixture(n_components=30, covariance_type='full', max_iter=100, random_state=0)
train_es = features_es[:int(len(features_es) * 3 / 4)]
sample_vectors = []
for vec in train_es:
    vec = np.swapaxes(vec,0,1)
    if not len(vec) < 75:
        vec = vec.reshape(75, 39)
        gmm_es = gmm_es.fit(vec)
        sample_vectors.append(vec)

shape = np.shape(sample_vectors)
sample_vectors = np.array(sample_vectors).reshape((shape[0] * shape[1]), shape[2])
gmm_es = gmm_es.fit(sample_vectors)

In [273]:
test_en = features_en[int(len(features_en) * 3 / 4) + 1:]
test_es = features_es[int(len(features_es) * 3 / 4) + 1:]
vec = test_es[12]
#print(vec)
#print(len(vec))
vec = np.swapaxes(vec,0,1)
if not len(vec) < 75:
    vec = vec.reshape(75, 39)
    pred_es = gmm_es.score(vec)
    pred_en = gmm_en.score(vec)

#show how the same vector gets different cluster sets
print(pred_en)
print(pred_es)

-90.53250421112527
-88.25573545834413


In [222]:
from collections import defaultdict
def make_cluster_dist(train_vectors, gmm):
    '''
    :param train_vectors: training vectors for each sample
    :gmm: gmm model used for training
    '''
    cluster_dist = defaultdict(float)
    for vec in train_vectors: #n_samples
        vec = np.swapaxes(vec,0,1)
        if not len(vec) < 75: #I should really change the 75 to a variable
            vec = vec.reshape(75, 39)
            for cluster in gmm.predict(vec): # 75 repetitions 
                cluster_dist[cluster] += 1
    return cluster_dist



In [223]:
clusters_dist_en = make_cluster_dist(train_en, gmm_en)
clusters_dist_es = make_cluster_dist(train_es, gmm_es)

In [224]:
import pprint as pp
total_clusters_en = sum(clusters_dist_en.values())
total_clusters_es = sum(clusters_dist_es.values())
clusters_prob_en = defaultdict(float)
clusters_prob_es = defaultdict(float)
for k in clusters_dist_en:
    clusters_prob_en[k] = clusters_dist_en[k] / total_clusters_en
    clusters_prob_es[k] = clusters_dist_es[k] / total_clusters_es
pp.pprint(clusters_prob_en)
pp.pprint(clusters_prob_es)

defaultdict(<class 'float'>,
            {0: 0.014600345647007993,
             1: 0.022904785050766905,
             2: 0.0293049254698639,
             3: 0.019050820911643982,
             4: 0.07232798660617844,
             5: 0.028184813134586303,
             6: 0.027146791963707066,
             7: 0.036269712680924604,
             8: 0.012612605314322747,
             9: 0.023206416072585872,
             10: 0.03339058111903219,
             11: 0.02948098941456038,
             12: 0.040017822423849644,
             13: 0.02965138258803197,
             14: 0.038668448909051634,
             15: 0.018343864765608123,
             16: 0.09080362929358393,
             17: 0.03431410671851372,
             18: 0.061123082739252536,
             19: 0.035164182328796714,
             20: 0.04244788291207604,
             21: 0.036950205227910995,
             22: 0.027558327932598832,
             23: 0.024569291423633614,
             24: 0.035100723698422986,
             25

In [225]:
gmm_en.predict_proba(vec)[74]

array([6.87610976e-012, 1.83350841e-091, 1.93252533e-025, 8.99225301e-150,
       2.15952519e-139, 7.06177624e-032, 4.88410910e-027, 1.89740301e-015,
       1.63580634e-059, 1.95095285e-007, 9.99958410e-001, 1.64622486e-012,
       4.09023695e-008, 3.31718549e-005, 4.94950900e-022, 3.33445260e-031,
       2.22572532e-084, 6.57206939e-011, 8.62279721e-046, 2.80488183e-016,
       2.79609452e-078, 3.91794360e-007, 4.32744147e-026, 7.78999306e-006,
       6.08016839e-042, 4.00272194e-016, 0.00000000e+000, 5.38833250e-036,
       2.16182534e-012, 2.75172867e-022])

In [276]:
def bays_predict(test_vec, models, dists):
    preds = []
    scores = []
    for gmm in models:
        #cluster predictions per sample per model
        preds.append(gmm.predict(test_vec))
        scores.append(gmm.score_samples(test_vec))
    #print(np.sum(scores[0]))
    prob_sum = []
    # sum(log(P(vec|z,lang)))
    for pred,score,dist in zip(preds, scores, dists):
        #print([dist[cluster] for cluster in pred])
        prob_sum.append(np.sum(score) + np.sum(np.log([dist[cluster] for cluster in pred if dist[cluster] > 0])))
    return prob_sum

In [251]:
# english_preds = []
# english_preds.append(bays_predict(vec, [gmm_en, gmm_es], [clusters_prob_en, clusters_prob_es]))
# print(english_preds)

In [277]:
english_preds = []
for vec in test_en:
    vec = np.swapaxes(vec,0,1)
    if not len(vec) < 75:
        vec = vec.reshape(75, 39)
        english_preds.append(np.argmax(bays_predict(vec, [gmm_en, gmm_es], [clusters_prob_en, clusters_prob_es])))

print(np.average(np.array(english_preds) == 0))

0.7914668932451295


In [278]:
spanish_preds = []
for vec in test_es:
    vec = np.swapaxes(vec,0,1)
    if not len(vec) < 75:
        vec = vec.reshape(75, 39)
        spanish_preds.append(np.argmax(bays_predict(vec, [gmm_en, gmm_es], [clusters_prob_en, clusters_prob_es])))

print(np.average(np.array(spanish_preds) == 1))

0.6101926026930288


In [279]:
import pickle
# Saving a model
with open('gmm_es','wb') as es_file:
    pickle.dump(gmm_es, es_file)
with open('gmm_en','wb') as en_file:
    pickle.dump(gmm_en, en_file)

In [280]:
# Loading saved model
gmm_es_ld = []
gmm_en_ld = []
with open('gmm_es','rb') as es_file:
    gmm_es_ld = pickle.load(es_file)
    
with open('gmm_en','rb') as en_file: 
    gmm_en_ld = pickle.load(en_file)


In [331]:
#validating models loaded are the same as saved models
print(gmm_en_ld.predict(vec))
print(np.shape(vec))


[ 4  4  4  4  4  4  4  4  4  4  4  4  4  4 16 16  2  2  2 16 16 16 16 16
 16 16 16 16  4  4  4 16  4  4  4  4  4  4  4  4 16  4  4  4 16  4 16  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  2  2  2  2  4  4  4  4  4  4 16
  4  4  4]
(75, 39)
